result:
    - model: papuGaPT2
        score: 0.585386
    - model: herbert
        score: 0.64205

zmienione słowa (herbert):
    - zamienione dwie ostatnie liter, score: 0.5
    - brak polskich znaków, score: 0.556882

In [ ]:
from transformers import AutoTokenizer, AutoModel

model_name = "allegro/herbert-base-cased"
device = 'cpu'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

text = 'Bardzo lubię lody malinowe z bitą śmietaną.'

token_ids = tokenizer(text, return_tensors='pt')['input_ids'][0]

print ([tokenizer.decode(idx) for idx in token_ids])

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['<s>', 'Bardzo', 'lubię', 'lody', 'mali', 'nowe', 'z', 'bi', 'tą', 'śmietan', 'ą', '.', '</s>']


In [112]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F
import random

model_name = 'flax-community/papuGaPT2'

device = 'cuda'
device = 'cpu'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [107]:
def tokenize(word, tokenizer):
    tokens = tokenizer(word, return_tensors='pt', add_special_tokens=False)
    return tokens['input_ids'][0]

In [108]:
def get_word_embedding(word, model, tokenizer):
    tokens_ids = tokenize(word, tokenizer=tokenizer)

    with torch.no_grad():
        embeddings = model.get_input_embeddings()(tokens_ids)
    word_embedding = embeddings.mean(dim=0).squeeze().numpy()
    # word_embedding = embeddings.sum(dim=0).squeeze().numpy()
    return word_embedding

In [109]:
print(tokenize('szczerbatek', tokenizer=tokenizer))

tensor([2542,   86, 2289, 2598])


In [110]:
print(get_word_embedding('szczerbatek', model=model, tokenizer=tokenizer))

[-1.05056763e-02  4.65011597e-02 -3.50341797e-02  4.46090698e-02
  2.14813232e-01  2.20361710e-01 -1.54800415e-01 -1.91610336e-01
 -7.85064697e-02 -4.65850830e-02 -9.65728760e-02  1.42227173e-01
 -2.55317688e-01  3.52153778e-02  1.80036545e-01  4.15611267e-03
  1.89422607e-01  3.90338898e-02  1.33148193e-01  2.03155518e-01
 -1.13456726e-01  1.19753361e-01  8.87222290e-02 -2.12734640e-01
  1.66976929e-01 -2.66929626e-01 -4.33253646e-02  1.42021179e-01
  1.94141388e-01  2.19421387e-01 -3.29345703e-01 -5.31768799e-02
  7.57980347e-02  1.73103333e-01 -1.57424927e-01  8.58383179e-02
 -1.43796921e-01  7.70530701e-02  3.39660645e-02 -2.23083496e-01
  9.23004150e-02 -3.54785919e-02  4.95232344e-02  5.29174805e-02
 -1.51883841e-01  3.37982178e-02 -1.78945541e-01  1.03576660e-01
 -2.72521973e-01  1.44330680e-01 -4.57916260e-02  2.08122253e-01
  8.69064331e-02  9.20104980e-03  2.33978271e-01  2.30064392e-01
 -4.94308472e-02  5.83190918e-02  1.62248611e-02 -9.17167664e-02
  2.60467529e-02  1.19079

In [41]:
from scipy.spatial.distance import cosine

In [43]:
def abx(a, b, c, model, tokenizer):
    emb_a = get_word_embedding(a, model, tokenizer)
    emb_b = get_word_embedding(b, model, tokenizer)
    emb_c = get_word_embedding(c, model, tokenizer)

    if cosine(emb_a, emb_b) < cosine(emb_a, emb_c):
        return "A jest bliżej B"
    else:
        return "A jest bliżej C"
    


In [45]:
result = abx("kot", "kot", "słoń", model, tokenizer)
print(result)

A jest bliżej B


In [75]:
clusters_txt = '''
piśmiennicze: pisak flamaster ołówek długopis pióro
małe_ssaki: mysz szczur chomik łasica kuna bóbr
okręty: niszczyciel lotniskowiec trałowiec krążownik pancernik fregata korweta
lekarze: lekarz pediatra ginekolog kardiolog internista geriatra
zupy: rosół żurek barszcz
uczucia: miłość przyjaźń nienawiść gniew smutek radość strach
działy_matematyki: algebra analiza topologia logika geometria 
budynki_sakralne: kościół bazylika kaplica katedra świątynia synagoga zbór
stopień_wojskowy: chorąży podporucznik porucznik kapitan major pułkownik generał podpułkownik
grzyby_jadalne: pieczarka borowik gąska kurka boczniak kania
prądy_filozoficzne: empiryzm stoicyzm racjonalizm egzystencjalizm marksizm romantyzm
religie: chrześcijaństwo buddyzm islam prawosławie protestantyzm kalwinizm luteranizm judaizm
dzieła_muzyczne: sonata synfonia koncert preludium fuga suita
cyfry: jedynka dwójka trójka czwórka piątka szóstka siódemka ósemka dziewiątka
owady: ważka biedronka żuk mrówka mucha osa pszczoła chrząszcz
broń_biała: miecz topór sztylet nóż siekiera
broń_palna: karabin pistolet rewolwer fuzja strzelba
komputery: komputer laptop kalkulator notebook
kolory: biel żółć czerwień błękit zieleń brąz czerń
duchowny: wikary biskup ksiądz proboszcz rabin pop arcybiskup kardynał pastor
ryby: karp śledź łosoś dorsz okoń sandacz szczupak płotka
napoje_mleczne: jogurt kefir maślanka
czynności_sportowe: bieganie skakanie pływanie maszerowanie marsz trucht
ubranie:  garnitur smoking frak żakiet marynarka koszula bluzka sweter sweterek sukienka kamizelka spódnica spodnie
mebel: krzesło fotel kanapa łóżko wersalka sofa stół stolik ława
przestępca: morderca zabójca gwałciciel złodziej bandyta kieszonkowiec łajdak łobuz 
mięso_wędliny: wieprzowina wołowina baranina cielęcina boczek baleron kiełbasa szynka schab karkówka dziczyzna
drzewo: dąb klon wiąz jesion świerk sosna modrzew platan buk cis jawor jarzębina akacja
źródło_światła: lampa latarka lampka żyrandol żarówka reflektor latarnia lampka
organ: wątroba płuco serce trzustka żołądek nerka macica jajowód nasieniowód prostata śledziona
oddziały: kompania pluton batalion brygada armia dywizja pułk
napój_alkoholowy: piwo wino wódka dżin nalewka bimber wiśniówka cydr koniak wiśniówka
kot_drapieżny: puma pantera lampart tygrys lew ryś żbik gepard jaguar
metal: żelazo złoto srebro miedź nikiel cyna cynk potas platyna chrom glin aluminium
samolot: samolot odrzutowiec awionetka bombowiec myśliwiec samolocik helikopter śmigłowiec
owoc: jabłko gruszka śliwka brzoskwinia cytryna pomarańcza grejpfrut porzeczka nektaryna
pościel: poduszka prześcieradło kołdra kołderka poduszeczka pierzyna koc kocyk pled
agd: lodówka kuchenka pralka zmywarka mikser sokowirówka piec piecyk piekarnik
'''
lines = clusters_txt.split('\n')
lines = [l.split(' ') for l in lines]
words = []
for l in lines:
    for w in l:
        if ':' not in w and w != '':
            words.append(w)


In [115]:
with open('word_embedings_file.txt', 'w') as output_file:
    for w in words:
        print(w, end=' ', file=output_file)
        emb = get_word_embedding(w, model, tokenizer)
        for i in emb:
            print(i, end=' ', file=output_file)
        print(file=output_file)